<a href="https://colab.research.google.com/github/abumafrim/Cohere-Align/blob/main/CoHere_Align.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Miscellaneous

##Installations

In [ ]:
import os

In [ ]:
home_dir = '/content/drive/MyDrive/CoHere'

if not os.path.isdir(home_dir):
  os.makedirs(home_dir)

%cd {home_dir}

if not os.path.isdir('Cohere-Align'):
  !git clone https://github.com/abumafrim/Cohere-Align.git
else:
  %cd Cohere-Align
  !git pull

%cd home_dir

/content/drive/MyDrive/CoHere
/content/drive/MyDrive/CoHere/Cohere-Align
Already up to date.
[Errno 2] No such file or directory: 'home_dir'
/content/drive/MyDrive/CoHere/Cohere-Align


In [ ]:
!pip install cohere laserembeddings --q

#download the pre-trained laser models:
!python -m laserembeddings download-models

In [ ]:
import nltk
from nltk.tokenize import sent_tokenize, word_tokenize
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

##Download FLORES

In [ ]:
# evaluation on flores devtest

# download flores
!wget --trust-server-names https://tinyurl.com/flores200dataset

!tar -xvzf flores200_dataset.tar.gz

/content/drive/MyDrive/CoHere/data
--2023-09-05 10:33:15--  https://tinyurl.com/flores200dataset
Resolving tinyurl.com (tinyurl.com)... 104.20.138.65, 172.67.1.225, 104.20.139.65, ...
Connecting to tinyurl.com (tinyurl.com)|104.20.138.65|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: https://dl.fbaipublicfiles.com/nllb/flores200_dataset.tar.gz [following]
--2023-09-05 10:33:16--  https://dl.fbaipublicfiles.com/nllb/flores200_dataset.tar.gz
Resolving dl.fbaipublicfiles.com (dl.fbaipublicfiles.com)... 52.84.251.15, 52.84.251.106, 52.84.251.27, ...
Connecting to dl.fbaipublicfiles.com (dl.fbaipublicfiles.com)|52.84.251.15|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 25585843 (24M) [application/x-tar]
Saving to: ‘flores200_dataset.tar.gz’

flores200_dataset.t 100%[===================>]  24.40M  78.8MB/s    in 0.3s    

2023-09-05 10:33:17 (78.8 MB/s) - ‘flores200_dataset.tar.gz’ saved [25585843/25585843]



## Download mafand

In [ ]:
!git clone https://github.com/masakhane-io/lafand-mt.git

Cloning into 'lafand-mt'...
remote: Enumerating objects: 1483, done.
remote: Counting objects: 100% (405/405), done.
remote: Compressing objects: 100% (375/375), done.
remote: Total 1483 (delta 89), reused 284 (delta 24), pack-reused 1078
Receiving objects: 100% (1483/1483), 115.42 MiB | 16.46 MiB/s, done.
Resolving deltas: 100% (301/301), done.
Updating files: 100% (1002/1002), done.


## Process crawled data

In [ ]:
data_dir = 'Cohere-Align/example/crawled_data'
langs = ['eng', 'hau']

def remove_empty_lines(text, clean=True):
  if clean:
    return [line for line in text if line.strip() and len(word_tokenize(line)) > 4 and len(word_tokenize(line)) < 81]
  else:
    return text

def retokenize_lines(lines):
  token_lines = []
  for line in lines:
    token_lines.extend(sent_tokenize(line))
  return token_lines

for lang in langs:
  data_path = os.path.join(data_dir, lang)
  files = os.listdir(data_path)
  merged_lines = set()

  try:
    for f in files:
      if f.endswith('.txt'):
        with open(os.path.join(data_path, f), 'r') as sfile:
          lines = retokenize_lines(sfile.read().splitlines())
          merged_lines.update(remove_empty_lines(lines))

    print("Merge operation completed successfully.")
    print(lang, 'number of sentences:', len(merged_lines))
    print('longest sentence length:', max(len(word_tokenize(line)) for line in merged_lines))
    print('shortest sentence length:', min(len(word_tokenize(line)) for line in merged_lines))

    with open(os.path.join(data_dir, lang + '.txt'), 'w') as merged_file:
      merged_file.writelines([str(f).strip() + '\n' for f in merged_lines])

  except Exception as e:
    print(f"An error occurred: {str(e)}")

  print()

Merge operation completed successfully.
eng number of sentences: 22671
longest sentence length: 80
shortest sentence length: 5

Merge operation completed successfully.
hau number of sentences: 13560
longest sentence length: 80
shortest sentence length: 5



#Evaulate Aligners

In [ ]:
%cd /content/drive/MyDrive/CoHere/Cohere-Align

##Generate alignment for MAFAND-MT using laser and cohere

In [ ]:
%cd /content/drive/MyDrive/CoHere

/content/drive/MyDrive/CoHere


In [ ]:
dts = 'dev,test,train'

for dt in dts.split(','):
  print(dt)
  !python3 Cohere-Align/scripts/laser_align.py \
    -s lafand-mt/data/text_files/en-hau/{dt}.hau \
    -t lafand-mt/data/text_files/en-hau/{dt}.en \
    -o Cohere-Align/data/aligned_data/mafand/{dt}/laser \
    --src_lang ha \
    --trg_lang en \
    --retrieval 'nn' \
    --dot \
    --cuda
  print()

dev
converted 1300 source embeddings to np matrix
converted 1300 target embeddings to np matrix
build source sent to index map: done
length of source embedding 1300
build target word to index map: done
length of target embedding 1300
1300 translations generated

test
converted 1500 source embeddings to np matrix
converted 1500 target embeddings to np matrix
build source sent to index map: done
length of source embedding 1500
build target word to index map: done
length of target embedding 1500
1500 translations generated

train
converted 3098 source embeddings to np matrix
converted 3098 target embeddings to np matrix
build source sent to index map: done
length of source embedding 3098
build target word to index map: done
length of target embedding 3098
3098 translations generated



In [ ]:
dts = 'dev,test,train'

for dt in dts.split(','):
  print(dt)
  !python3 Cohere-Align/scripts/cohere_align.py \
    --cohere_api_key 'lT27lL4uyB6e8KnTl9tRUDjF1TMksqDWKL8oDjHU' \
    -m 'embed-multilingual-v2.0' \
    -s lafand-mt/data/text_files/en-hau/{dt}.hau \
    -t lafand-mt/data/text_files/en-hau/{dt}.en \
    -o Cohere-Align/data/aligned_data/mafand/{dt}/cohere \
    --retrieval 'nn' \
    --dot \
    --cuda
  print()

dev
reading sentences and embeddings from file
build source sent to index map: done
length of source embedding 1300
build target word to index map: done
length of target embedding 1300

test
reading sentences and embeddings from file
build source sent to index map: done
length of source embedding 1500
build target word to index map: done
length of target embedding 1500

train
reading sentences and embeddings from file
build source sent to index map: done
length of source embedding 3098
build target word to index map: done
length of target embedding 3098



##Generate alignment for FLORES using laser and cohere

In [ ]:
%cd /content/drive/MyDrive/CoHere

/content/drive/MyDrive/CoHere


In [ ]:
dts = 'dev,devtest'

for dt in dts.split(','):
  print(dt)
  !python3 Cohere-Align/scripts/laser_align.py \
    -s flores200_dataset/{dt}/hau_Latn.{dt} \
    -t flores200_dataset/{dt}/eng_Latn.{dt} \
    -o Cohere-Align/data/aligned_data/flores/{dt}/laser \
    --src_lang ha \
    --trg_lang en \
    --retrieval 'nn' \
    --dot \
    --cuda
  print()

dev
converted 1300 source embeddings to np matrix
converted 1300 target embeddings to np matrix
build source sent to index map: done
length of source embedding 1300
build target word to index map: done
length of target embedding 1300
1300 translations generated

test
converted 1500 source embeddings to np matrix
converted 1500 target embeddings to np matrix
build source sent to index map: done
length of source embedding 1500
build target word to index map: done
length of target embedding 1500
1500 translations generated

train
converted 3098 source embeddings to np matrix
converted 3098 target embeddings to np matrix
build source sent to index map: done
length of source embedding 3098
build target word to index map: done
length of target embedding 3098
3098 translations generated



In [ ]:
dts = 'dev,test,train'

for dt in dts.split(','):
  print(dt)
  !python3 Cohere-Align/scripts/cohere_align.py \
    --cohere_api_key 'lT27lL4uyB6e8KnTl9tRUDjF1TMksqDWKL8oDjHU' \
    -m 'embed-multilingual-v2.0' \
    -s flores200_dataset/{dt}/hau_Latn.{dt} \
    -t flores200_dataset/{dt}/eng_Latn.{dt} \
    -o Cohere-Align/data/aligned_data/flores/{dt}/cohere \
    --retrieval 'nn' \
    --dot \
    --cuda
  print()

dev
reading sentences and embeddings from file
build source sent to index map: done
length of source embedding 1300
build target word to index map: done
length of target embedding 1300

test
reading sentences and embeddings from file
build source sent to index map: done
length of source embedding 1500
build target word to index map: done
length of target embedding 1500

train
reading sentences and embeddings from file
build source sent to index map: done
length of source embedding 3098
build target word to index map: done
length of target embedding 3098



##Calculate F1 score

In [ ]:
from sklearn.metrics import f1_score
import pandas as pd

In [ ]:
d = 'mafand'
dts = 'dev,test,train'

for dt in dts.split(','):
  dp = 'lafand-mt/data/text_files/en-hau/' + dt + '.en'
  with open(dp) as f:
    y_true = f.readlines()
    y_true = [y.strip() for y in y_true]

  # cohere f1
  df = pd.read_csv('data/cohere/' + d + '/' + dt + '/cohere_translations.csv')
  y_pred = df['translations'].tolist()
  cf1 = f1_score(y_true, y_pred, average='macro')

  # flores f1
  df = pd.read_csv('data/laser/' + d + '/' + dt + '/laser_translations.csv')
  y_pred = df['translations'].tolist()
  lf1 = f1_score(y_true, y_pred, average='macro')

  print(d, dt, '&', len(y_true), '&', str(round(lf1 * 100, 2)) + '\\%', '&', str(round(cf1 * 100, 2)) + '\\%', '\\\\')

mafand dev & 1300 & 0.64\% & 49.19\% \\
mafand test & 1500 & 0.43\% & 54.83\% \\
mafand train & 3098 & 0.33\% & 39.27\% \\


#Generate Aligned Sentences for PT Crawled data

In [ ]:
%cd /content/drive/MyDrive/CoHere/Cohere-Align

## Generate alignment for PT data

In [ ]:
!python3 scripts/laser_align.py \
  -s data/crawled_data/hau.txt \
  -t data/crawled_data/eng.txt \
  -o data/aligned_data/pt/laser \
  --src_lang ha \
  --trg_lang en \
  --retrieval 'nn' \
  --dot \
  --cuda

build source sent to index map: done
length of source embedding 13560
build target word to index map: done
length of target embedding 22671
13560 translations generated


In [ ]:
!python3 Cohere-Align/scripts/cohere_align.py \
    --cohere_api_key 'lT27lL4uyB6e8KnTl9tRUDjF1TMksqDWKL8oDjHU' \
    -m 'embed-multilingual-v2.0' \
    -s data/crawled_data/hau.txt \
    -t data/crawled_data/eng.txt \
    -o data/aligned_data/pt/cohere \
    --retrieval 'nn' \
    --dot \
    --cuda

Due to Cohere free API limitations, embeddings of batches will be generated after 61 seconds of each other.
batch 1 of 12 done - 2000 source and 2000 target embeddings generated successfully.

Sleeping: 100% 61/61 [01:01<00:00,  1.00s/it]
batch 2 of 12 done - 2000 source and 2000 target embeddings generated successfully.

Sleeping: 100% 61/61 [01:01<00:00,  1.00s/it]
batch 3 of 12 done - 2000 source and 2000 target embeddings generated successfully.

Sleeping: 100% 61/61 [01:01<00:00,  1.00s/it]
batch 4 of 12 done - 2000 source and 2000 target embeddings generated successfully.

Sleeping: 100% 61/61 [01:01<00:00,  1.00s/it]
batch 5 of 12 done - 2000 source and 2000 target embeddings generated successfully.

Sleeping: 100% 61/61 [01:01<00:00,  1.00s/it]
batch 6 of 12 done - 2000 source and 2000 target embeddings generated successfully.

Sleeping: 100% 61/61 [01:01<00:00,  1.00s/it]
batch 7 of 12 done - 1560 source and 2000 target embeddings generated successfully.

Sleeping: 100% 61/61 

##Process the aligned sentences

In [ ]:
%cd '/content/drive/MyDrive/CoHere'

import pandas as pd

/content/drive/MyDrive/CoHere


In [ ]:
method = 'cohere,laser'

for m in method.split(','):
  print(m)
  p = 'Cohere-Align/data/aligned_data/pt/' + m

  df = pd.read_csv(p + '/translations.csv')
  print('before removing duplicate translations', len(df))

  df = df.drop_duplicates(subset='translations', keep='first')
  print('after removing duplicate translations', len(df))

  df.to_csv(p + '/translations_dedup.csv', index=False)
  print()

cohere
before removing duplicate translations 13560
after removing duplicate translations 3722

laser
before removing duplicate translations 13560
after removing duplicate translations 400



##Create samples for human evaluation

In [ ]:
evaluators = ['babangida', 'bala', 'lukman']
sents = 25
method = {'cohere': 2, 'laser': 1}

p = 'Cohere-Align/data/aligned_data/pt/' + m

for m, n in method.items():
  df = pd.read_csv(p + '/translations_dedup.csv')

  # shuffle translations
  df = df.sample(frac=1.0)

  for i, e in enumerate(evaluators):
    evaluator = df.iloc[i * sents : i * sents + sents]
    evaluator.to_csv('Cohere-Align/human_eval/' + str(n) + '_' + e + '_dedup.csv', index=False)

In [ ]:
p = '/content/drive/MyDrive/CoHere/Cohere-Align/human_eval'
import os

files = os.listdir(p)

for f in files:
  if f.endswith('.csv'):
    pt = os.path.join(p, f)
    !wc -l {pt}

26 /content/drive/MyDrive/CoHere/data/evaluation/1_babangida.csv
26 /content/drive/MyDrive/CoHere/data/evaluation/1_bala.csv
26 /content/drive/MyDrive/CoHere/data/evaluation/1_lukman.csv
26 /content/drive/MyDrive/CoHere/data/evaluation/2_babangida.csv
26 /content/drive/MyDrive/CoHere/data/evaluation/2_bala.csv
26 /content/drive/MyDrive/CoHere/data/evaluation/2_lukman.csv


##Update the repo with the corrections to the code

In [ ]:
%cd /content/drive/MyDrive/CoHere/Cohere-Align

!git config user.email "abumafrim@gmail.com"
!git config user.name "abumafrim"

/content/drive/MyDrive/CoHere/Cohere-Align


In [ ]:
# update repo
!git add --all
!git commit -m "prepared repo for publication"
!git push origin main

/content/drive/MyDrive/CoHere/Cohere-Align
[main 1b45507] prepared repo for publication
 2009 files changed, 76916 insertions(+), 10 deletions(-)
 delete mode 100644 cohere/translations.csv
 create mode 100644 data/crawled_data/eng.txt
 create mode 100644 data/crawled_data/eng/1.txt
 create mode 100644 data/crawled_data/eng/10.txt
 create mode 100644 data/crawled_data/eng/100.txt
 create mode 100644 data/crawled_data/eng/1000.txt
 create mode 100644 data/crawled_data/eng/101.txt
 create mode 100644 data/crawled_data/eng/102.txt
 create mode 100644 data/crawled_data/eng/103.txt
 create mode 100644 data/crawled_data/eng/104.txt
 create mode 100644 data/crawled_data/eng/105.txt
 create mode 100644 data/crawled_data/eng/106.txt
 create mode 100644 data/crawled_data/eng/107.txt
 create mode 100644 data/crawled_data/eng/108.txt
 create mode 100644 data/crawled_data/eng/109.txt
 create mode 100644 data/crawled_data/eng/11.txt
 create mode 100644 data/crawled_data/eng/110.txt
 create mode 1006

##Create stats for graphs

In [ ]:
import pandas as pd
import warnings
warnings.filterwarnings('ignore')

df1 = pd.read_csv('data/aligned_data/pt/laser/translations.csv', header=0, names=['ha', 'enl'])
df1.head()

,ha,enl
0,Malam Uba Sani ne zai kara da Isah Ashiru na P...,"Others are Jessica Aldred, Special projects ed..."
1,Okasanmi ya ce tuni rundunar ta fara farautar ...,"The first count alleged that Seplat Energy, it..."
2,Faston da ya halasta auren mace fiye da ɗaya a...,Others are Magnesium trisilicate tablets and s...
3,Amai: Saboda zafin zazzabi da ciwon kai mutum ...,Other great historic members include local and...
4,Mai martaba Sarkin Dutse ya yi shekaru ashirin...,"According to INEC, Mr Tinubu scored 25 per cen..."


In [ ]:
df2 = pd.read_csv('data/aligned_data/pt/cohere/translations.csv', header=0, names=['ha', 'enc'])
df2.head()

,ha,enc
0,Malam Uba Sani ne zai kara da Isah Ashiru na P...,He will be competing against Deputy Speaker Id...
1,Okasanmi ya ce tuni rundunar ta fara farautar ...,"The Director, Defence Media Operations, Musa D..."
2,Faston da ya halasta auren mace fiye da ɗaya a...,The complainant who resides in Rigasa area of ...
3,Amai: Saboda zafin zazzabi da ciwon kai mutum ...,• Nasir Sani Zangon Daura – Katsina North
4,Mai martaba Sarkin Dutse ya yi shekaru ashirin...,The 71-year-old is widely seen as Nigeria’s po...


In [ ]:
merged_df = pd.merge(df1, df2, on='ha', how='left')
merged_df.head()

,ha,enl,enc
0,Malam Uba Sani ne zai kara da Isah Ashiru na P...,"Others are Jessica Aldred, Special projects ed...",He will be competing against Deputy Speaker Id...
1,Okasanmi ya ce tuni rundunar ta fara farautar ...,"The first count alleged that Seplat Energy, it...","The Director, Defence Media Operations, Musa D..."
2,Faston da ya halasta auren mace fiye da ɗaya a...,Others are Magnesium trisilicate tablets and s...,The complainant who resides in Rigasa area of ...
3,Amai: Saboda zafin zazzabi da ciwon kai mutum ...,Other great historic members include local and...,• Nasir Sani Zangon Daura – Katsina North
4,Mai martaba Sarkin Dutse ya yi shekaru ashirin...,"According to INEC, Mr Tinubu scored 25 per cen...",The 71-year-old is widely seen as Nigeria’s po...


In [ ]:
merged_df['ha_lengths'] = merged_df['ha'].apply(lambda x: len(word_tokenize(x)))
merged_df['enl_lengths'] = merged_df['enl'].apply(lambda x: len(word_tokenize(x)))
merged_df['enc_lengths'] = merged_df['enc'].apply(lambda x: len(word_tokenize(x)))

sc = ['ha_lengths', 'enl_lengths', 'enc_lengths']

new_df = merged_df[sc]

new_df.head()

,ha_lengths,enl_lengths,enc_lengths
0,25,80,67
1,14,63,29
2,22,59,34
3,16,62,8
4,44,70,43


In [ ]:
for s in sc:
  new_df[s] = new_df[s] / 10

new_df.head()

<ipython-input-63-dc95551e6d53>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  new_df[s] = new_df[s] / 10


,ha_lengths,enl_lengths,enc_lengths
0,2.5,8.0,6.7
1,1.4,6.3,2.9
2,2.2,5.9,3.4
3,1.6,6.2,0.8
4,4.4,7.0,4.3


In [ ]:
new_df = new_df.sort_values(by='ha_lengths')
new_df = new_df.reset_index(drop=True)
print(len(new_df))
new_df.head()

13562


,ha_lengths,enl_lengths,enc_lengths
0,0.5,6.8,0.5
1,0.5,0.5,0.8
2,0.5,7.2,0.7
3,0.5,5.2,0.7
4,0.5,3.9,0.7


In [ ]:
scl = new_df.sample(n=20)

scl.insert(0, 'sents', [f'sent\\_{i}' for i in range(1, len(scl) + 1)])
scl.head()

,sents,ha_lengths,enl_lengths,enc_lengths
7547,sent\_1,2.6,6.2,2.4
4703,sent\_2,2.0,7.0,3.9
4518,sent\_3,2.0,8.0,2.1
13121,sent\_4,5.1,8.0,3.4
4284,sent\_5,2.0,7.5,2.8


In [ ]:
scl.to_csv('src_tgt_lengths.tsv', sep='\t', index=False)

In [ ]:
!head /content/drive/MyDrive/CoHere/data/src_tgt_lengths.tsv

sents	ha_lengths	enl_lengths	enc_lengths
sent\_1	2.6	6.2	2.4
sent\_2	2.0	7.0	3.9
sent\_3	2.0	8.0	2.1
sent\_4	5.1	8.0	3.4
sent\_5	2.0	7.5	2.8
sent\_6	2.4	4.8	2.6
sent\_7	2.6	5.9	3.4
sent\_8	2.7	8.0	3.1
sent\_9	2.9	7.9	2.9
